# PE Resolution Classification

We perform a univariable classifications using each explanatory variable. We then perform multivariable classifications. For these, we focus on body composition only, cardiopulmonary features only, and then a composite model. For each of these, we perform three forms of feature selection, using (1) recursive feature elimination with cross validation, (2) forward sequential feature selection with cross validation, and (3) backward feature selection with cross validation. For these groups of selected features, we also perform sensitivities controlling for gender, age, and both gender and age.

In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from pathlib import Path
import os
import pickle
from tqdm.notebook import trange, tqdm
from config import model_config

from scipy.stats import shapiro
from sklearn.ensemble import (
	RandomForestClassifier
)
from sklearn.feature_selection import(
	RFECV, SequentialFeatureSelector
)
from sklearn.linear_model import (
	LinearRegression, LogisticRegression,
    LogisticRegressionCV
)
from sklearn.metrics import (
	confusion_matrix, classification_report, f1_score,
	roc_curve, roc_auc_score, auc, RocCurveDisplay
)
from sklearn.model_selection import (
	train_test_split, RandomizedSearchCV, GridSearchCV, 
	cross_val_score, cross_val_predict, KFold, StratifiedKFold
)
from sklearn.pipeline import (
	Pipeline
)
from sklearn.preprocessing import (
	LabelEncoder, OneHotEncoder, StandardScaler,
	RobustScaler, QuantileTransformer,
)
import statsmodels.api as sm
from xgboost import XGBClassifier

from regression import reg

In [61]:
SEED = 123
TEST_SIZE = 0.25
CV_FOLDS = 5

HEATMAP_COLORS = sns.diverging_palette(h_neg=359, h_pos=250, as_cmap=True)
plt.style.use('ggplot')

USE_INITIAL = True
USE_CLUSTERED_SE = False

# Import Data

In [62]:
if USE_INITIAL:
    with open(Path('../data/classification_data_initial.pkl'), 'rb') as f:
    	data = pickle.load(f)
else:
    with open(Path('../data/classification_data_all.pkl'), 'rb') as f:
    	data = pickle.load(f)
	
X = data.get('X')
y = data.get('y').squeeze()
body_features = data.get('body_features')
cardio_features = data.get('cardio_features')
control_features = data.get('controls')
all_features = body_features + cardio_features + control_features

print(X.shape)
print(y.shape)
print(body_features)
print(cardio_features)
print(control_features)

(44, 37)
(44,)
['volume_visceral_fat', 'density_visceral_fat', 'mass_visceral_fat', 'volume_subcutaneous_fat', 'density_subcutaneous_fat', 'mass_subcutaneous_fat', 'volume_intermuscular_fat', 'density_intermuscular_fat', 'mass_intermuscular_fat', 'volume_muscle', 'density_muscle', 'mass_muscle', 'volume_bone', 'density_bone', 'mass_bone', 'bmi', 'bsa']
['emphysema_volume_950hu', 'lung_volume', 'extrapulmonary_artery_volume', 'extrapulmonary_vein_volume', 'intrapulmonary_artery_volume', 'intrapulmonary_vein_volume', 'artery_vein_ratio', 'bv5', 'bv10', 'pb_larger_10', 'pv_diameter', 'a_diameter', 'pv_a', 'heart_volume', 'airway_volume', 'airway_ratio', 'ild_volume', 'ild_ratio']
['age', 'gender_cl_Male']


In [63]:
pe_numbers = y.index.str[:-2]
print(len(pe_numbers))
pe_numbers

44


Index(['PE1', 'PE12', 'PE14', 'PE15', 'PE16', 'PE17', 'PE18', 'PE19', 'PE2',
       'PE20', 'PE21', 'PE22', 'PE23', 'PE24', 'PE25', 'PE27', 'PE28', 'PE3',
       'PE31', 'PE32', 'PE33', 'PE34', 'PE36', 'PE37', 'PE4', 'PE40', 'PE41',
       'PE42', 'PE43', 'PE44', 'PE45', 'PE47', 'PE48', 'PE49', 'PE5', 'PE51',
       'PE52', 'PE53', 'PE54', 'PE56', 'PE6', 'PE7', 'PE8', 'PE9'],
      dtype='object')

In [64]:
y.head(2)

PE1_0     1
PE12_0    0
Name: resolved_pe, dtype: int64

In [65]:
y.value_counts(dropna=False)

0    28
1    16
Name: resolved_pe, dtype: int64

# Logit Regression Functions

In [66]:
def get_params(model, X, y):
    """Returns pd.Series of coefs for comparison with statsmodels params."""
    y = np.array(y).ravel()
    model.fit(X, y)
    coef = pd.Series(np.squeeze(model.coef_), index=np.squeeze(model.feature_names_in_))
    # coef['const'] = model.intercept_
    return coef.sort_index()

In [67]:
def backward_stepwise_selection(X, y, cutoff):
    # Make copies of X, y
    X_temp = sm.add_constant(X.copy())
    y_temp = y.copy()
    
    # Fit initial model
    if USE_CLUSTERED_SE:
        model_sm = sm.Logit(y_temp, X_temp).fit(cov_type='cluster', disp=False, cov_kwds={'groups': pe_numbers})
    else: 
        model_sm = sm.Logit(y_temp, X_temp).fit(cov_type='HC3', disp=False)
        
        
    coefs = model_sm.params[1:]
    pvals = model_sm.pvalues[1:]
    df_temp = pd.DataFrame({
        'coefs': coefs,
        'pvals': pvals
    })
    current_varlist = list(coefs.index.values)

    # Store progression in a list of lists
    progression = list()
    progression.append(dict(zip(coefs.index.values, zip(coefs.values, pvals.values))))
    
    # Iterate until all are stat signif
    while not np.all(df_temp['pvals'] < cutoff):
        
        # Drop the variable with the highest pvalue
        new_vars = df_temp.drop(index=df_temp['pvals'].idxmax()).index.values
        
        # If remaining varlist is empty, break and return the last regression results
        if len(new_vars) == 0:
            break

        # Subset X to new list of variables
        X_temp = sm.add_constant(X_temp.loc[:, new_vars])
        
        # Re-fit model
        if USE_CLUSTERED_SE:
            model_sm = sm.Logit(y_temp, X_temp).fit(cov_type='cluster', disp=False, cov_kwds={'groups': pe_numbers})
        else: 
            model_sm = sm.Logit(y_temp, X_temp).fit(cov_type='HC3', disp=False)
            
        coefs = model_sm.params[1:]
        pvals = model_sm.pvalues[1:]
        df_temp = pd.DataFrame({
            'coefs': coefs,
            'pvals': pvals
        })
        progression.append(dict(zip(coefs.index.values, zip(coefs.values, pvals.values))))
        current_varlist = [var for var in model_sm.params.index.values if var != 'const']
    
    return current_varlist, progression

# Example
feat_out, prog = backward_stepwise_selection(X[body_features], y, 0.05)
print(feat_out)

['volume_visceral_fat', 'density_visceral_fat', 'mass_visceral_fat']


In [68]:
def model_residual_correlation(model):
    """Returns measure of correlation."""
    return np.corrcoef(np.arange(len(model.resid)), model.resid)[1, 0]

In [69]:
def fit_model(X, y):
    """Fit statsmodels OLS model with robust SEs and sklearn OLS model."""
    # Fit statsmodels model for pvalues and coef
    
    if USE_CLUSTERED_SE:
        model_sm = sm.Logit(y, X).fit(cov_type='cluster', disp=False, cov_kwds={'groups': pe_numbers})
    else: 
        model_sm = sm.Logit(y, X).fit(cov_type='HC3', disp=False)

    # Define sklearn model for CV evaluation
    model_sk = LogisticRegression(
        random_state=SEED,
        fit_intercept=False,
        max_iter=10_000, 
        tol=0.000001,
        penalty=None, 
        solver='newton-cg',
    )
    # Check that model params match
    # print(get_params(model_sk, X, y))
    # print(model_sm.params.sort_values())
    # print(np.isclose(get_params(model_sk, X, y), model_sm.params.sort_values()))
    assert np.all(np.isclose(get_params(model_sk, X, y), model_sm.params.sort_index()))
    model_sk.fit(X, y)
    return model_sm, model_sk

In [70]:
def store_model_results(model_sm, model_sk, X, y):
    """
    Params:
        - model_sm: statsmodel model for coefs, pvalues, and residuals.
        - model_sk: sklearn model for cross validation
        - X: X data.
        - y: y data.
    """
    # Calculate CV scores
    cv_scores = cross_val_score(
        model_sk, X, y, 
        scoring='roc_auc', 
        cv=CV_FOLDS, n_jobs=-1
    )
    # Store model results
    model_results = pd.DataFrame(
        {
            'y': y.name,
            'model_dfn': [tuple(X.columns.values)],
            'nobs': model_sm.nobs,
            'shapiro_resid_pvalue': np.nan,
            'metric_train': model_sk.score(X, y),
            'metric_cv_mean': np.mean(np.abs(cv_scores)),
            'metric_cv_std': np.std(cv_scores),
        }
    )
    # Set model index
    model_results = model_results.set_index(['y', 'model_dfn'])
    return model_results

In [71]:
def store_coef_results(model_sm, y):
    """
    Params:
        - model_sm: statsmodel model for coefs, pvalues, and residuals.
        - y: y data.
    """
    results = pd.DataFrame(
        {
            'model_dfn': [tuple(model_sm.params.index) for _ in range(len(model_sm.params))],
            'coef': model_sm.params, 
            'pval': model_sm.pvalues,
        },
    )
    results['signif'] = results['pval'].apply(reg.add_significance)
    results = results.reset_index(names='x')
    results['y'] = y.name
    results = results.pivot(index=['y', 'model_dfn'], columns=['x'], values=['coef', 'pval', 'signif'])
    results.columns = ['_'.join(idx) for idx in results.columns]
    return results

In [72]:
def combine_model_results(model_sm, model_sk, X, y):
    model_results = store_model_results(model_sm, model_sk, X, y)
    coef_results = store_coef_results(model_sm, y)
    assert model_results.shape[0] == coef_results.shape[0] 
    combined_results = pd.concat([model_results, coef_results], axis=1)
    return combined_results

## Example

In [73]:
target = 'resolved_pe'
features = 'pv_a'
X_temp = sm.add_constant(X[features])
y_temp = y.copy()
model_sm, model_sk = fit_model(X_temp, y_temp)

In [74]:
store_model_results(model_sm, model_sk, X_temp, y_temp)

,,nobs,shapiro_resid_pvalue,metric_train,metric_cv_mean,metric_cv_std
y,model_dfn,,,,,
resolved_pe,"(const, pv_a)",44,NaN,0.659091,0.346667,0.221041


In [75]:
store_coef_results(model_sm, y_temp)

,,coef_const,coef_pv_a,pval_const,pval_pv_a,signif_const,signif_pv_a
y,model_dfn,,,,,,
resolved_pe,"(const, pv_a)",-0.616329,-0.292203,0.063157,0.384242,,


In [76]:
model_sk.predict(sm.add_constant(X[features]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)

In [77]:
target = 'resolved_pe'
# features = ['pv_a']
# features = ['density_bone', 'pv_a']
# features = ['density_bone', 'a_diameter', 'pv_a', 'age', 'gender_cl_Male']
features = ['density_bone', 'pv_a', 'gender_cl_Male']

X_temp = sm.add_constant(X[features])
y_temp = y.copy()

# Statsmodels
if USE_CLUSTERED_SE:
    model = sm.Logit(y_temp, X_temp).fit(cov_type='cluster', disp=False, cov_kwds={'groups': pe_numbers})
else: 
    model = sm.Logit(y_temp, X_temp).fit(cov_type='HC3', disp=False)
    
print(pd.DataFrame({"coefs": model.params, "pvals": model.pvalues}))

# Sklearn 
model_sk = LogisticRegression(
        random_state=SEED,
        fit_intercept=False,
        max_iter=10_000, 
        tol=0.000001,
        penalty=None, 
        solver='newton-cg',
)
model_sk.fit(X_temp, y_temp)
cv_scores = cross_val_score(model_sk, X_temp, y_temp, scoring='roc_auc', cv=CV_FOLDS, n_jobs=-1)
print(model_sk.coef_)
print(np.mean(cv_scores))

                   coefs     pvals
const          -0.855998  0.031785
density_bone   -0.504049  0.098417
pv_a           -0.125301  0.730546
gender_cl_Male  0.720748  0.029200
[[-0.85599766 -0.50404927 -0.12530072  0.72074753]]
0.5866666666666667


# Perform univariable regressions 

In [78]:
univariate_models = pd.DataFrame()
univariate_coefs = pd.DataFrame()

for feature in tqdm(all_features):
    # Fit model
    X_temp = sm.add_constant(X[feature])
    y_temp = y.copy()
    try:
        model_sm, model_sk = fit_model(X_temp, y_temp)
    except:
        print(f"Logit on {feature} failed to converge")
        continue
    # Collect model information
    univariate_models = pd.concat([univariate_models, store_model_results(model_sm, model_sk, X_temp, y_temp)], 
                                  axis=0)

    # Collect coef information
    univariate_coefs = pd.concat([univariate_coefs, store_coef_results(model_sm, y_temp)], 
                                 axis=0)
    
print(univariate_models.shape)
print(univariate_coefs.shape)

  0%|          | 0/37 [00:00<?, ?it/s]

(37, 5)
(37, 114)


In [79]:
univariate_results = univariate_models.join(univariate_coefs, how='left', validate='1:1')
univariate_results = univariate_results.reset_index()
univariate_results['selection_method'] = 'All'
univariate_results['model_dfn'] = univariate_results['model_dfn'].apply(lambda x: x[1])
univariate_results['category'] = 'univariable_' + univariate_results['model_dfn']
univariate_results['controls'] = 'None'
univariate_results.index = univariate_results[['category', 'selection_method', 'y', 'controls']].apply('%'.join, axis=1)
univariate_results.index.name = 'Lookup'
print(univariate_results.shape)
univariate_results

(37, 124)


,y,model_dfn,nobs,shapiro_resid_pvalue,metric_train,metric_cv_mean,metric_cv_std,coef_const,coef_volume_visceral_fat,pval_const,...,signif_ild_ratio,coef_age,pval_age,signif_age,coef_gender_cl_Male,pval_gender_cl_Male,signif_gender_cl_Male,selection_method,category,controls
Lookup,,,,,,,,,,,,,,,,,,,,,
univariable_volume_visceral_fat%All%resolved_pe%None,resolved_pe,volume_visceral_fat,44,NaN,0.613636,0.612222,0.132981,-0.582495,0.125076,0.063812,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_volume_visceral_fat,None
univariable_density_visceral_fat%All%resolved_pe%None,resolved_pe,density_visceral_fat,44,NaN,0.636364,0.628889,0.155928,-0.676904,NaN,0.036075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_density_visceral_fat,None
univariable_mass_visceral_fat%All%resolved_pe%None,resolved_pe,mass_visceral_fat,44,NaN,0.613636,0.612222,0.132981,-0.582498,NaN,0.063805,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_mass_visceral_fat,None
univariable_volume_subcutaneous_fat%All%resolved_pe%None,resolved_pe,volume_subcutaneous_fat,44,NaN,0.636364,0.435556,0.168274,-0.561734,NaN,0.073363,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_volume_subcutaneous_fat,None
univariable_density_subcutaneous_fat%All%resolved_pe%None,resolved_pe,density_subcutaneous_fat,44,NaN,0.636364,0.452222,0.113725,-0.563769,NaN,0.072595,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_density_subcutaneous_fat,None
univariable_mass_subcutaneous_fat%All%resolved_pe%None,resolved_pe,mass_subcutaneous_fat,44,NaN,0.636364,0.435556,0.168274,-0.561847,NaN,0.073319,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_mass_subcutaneous_fat,None
univariable_volume_intermuscular_fat%All%resolved_pe%None,resolved_pe,volume_intermuscular_fat,44,NaN,0.636364,0.366667,0.122525,-0.566334,NaN,0.071761,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_volume_intermuscular_fat,None
univariable_density_intermuscular_fat%All%resolved_pe%None,resolved_pe,density_intermuscular_fat,44,NaN,0.636364,0.190000,0.113388,-0.569054,NaN,0.075936,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_density_intermuscular_fat,None
univariable_mass_intermuscular_fat%All%resolved_pe%None,resolved_pe,mass_intermuscular_fat,44,NaN,0.636364,0.366667,0.122525,-0.566158,NaN,0.071834,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_mass_intermuscular_fat,None


# Perform multivariable regressions

In [80]:
feature_options = {
    'body': body_features,
    'cardio': cardio_features,
    'composite': body_features + cardio_features,
}

## Feature selection

### Ensuring statistical significance

### Lasso Regularization

In [87]:
X_all = X[all_features]
logit = LogisticRegressionCV(cv=10, penalty='l1', solver='liblinear', max_iter=10_000, scoring='roc_auc')
logit.fit(X_all, y)

auc = cross_val_score(
    logit, 
    X_all,
    y,
    scoring='roc_auc',
    cv=10
)

print(f"ROC AUC: {np.mean(auc)}")

coefs = pd.DataFrame(
    {'coef': np.squeeze(logit.coef_)},
    index=logit.feature_names_in_
)

remaining_vars = coefs[coefs['coef'] != 0].index.values
remaining_vars

ROC AUC: 0.4833333333333334


array(['volume_visceral_fat', 'density_visceral_fat', 'mass_visceral_fat',
       'volume_subcutaneous_fat', 'density_subcutaneous_fat',
       'mass_subcutaneous_fat', 'volume_intermuscular_fat',
       'density_intermuscular_fat', 'mass_intermuscular_fat',
       'volume_muscle', 'density_muscle', 'mass_muscle', 'volume_bone',
       'density_bone', 'mass_bone', 'bmi', 'bsa',
       'emphysema_volume_950hu', 'lung_volume',
       'extrapulmonary_artery_volume', 'extrapulmonary_vein_volume',
       'intrapulmonary_artery_volume', 'intrapulmonary_vein_volume',
       'artery_vein_ratio', 'bv10', 'pb_larger_10', 'pv_diameter',
       'a_diameter', 'pv_a', 'heart_volume', 'airway_volume',
       'airway_ratio', 'ild_volume', 'ild_ratio', 'age', 'gender_cl_Male'],
      dtype=object)

In [108]:
curr_best = 0
curr_auc = 1e-6

remaining_features = all_features

while curr_auc > curr_best:
    X_temp = X[remaining_features]
    logit = LogisticRegressionCV(cv=10, penalty='l1', solver='liblinear', max_iter=10_000, scoring='roc_auc')
    logit.fit(X_temp, y)
    
    auc = cross_val_score(
        logit, 
        X_temp,
        y,
        scoring='roc_auc',
        cv=10
    )
    curr_auc = np.mean(auc)
    if curr_best < curr_auc:
        curr_best = curr_auc
        curr_auc = curr_best + 1e-6
    
    coefs = pd.DataFrame(
        {'coef': np.squeeze(logit.coef_)},
        index=logit.feature_names_in_
    )
    remaining_features = coefs[coefs['coef'] != 0].index.values
    # print(f"ROC AUC: {curr_auc}", remaining_features)

print(remaining_features)

# Fit model
X_temp = sm.add_constant(X[remaining_features])
y_temp = y.copy()
try:
    model_sm, model_sk = fit_model(X_temp, y_temp)
except:
    print(f"Logit on {feature} failed to converge")
# Collect model information
multivariable_models = pd.concat(
    [store_model_results(model_sm, model_sk, X_temp, y_temp), store_coef_results(model_sm, y_temp)], 
    axis=1
)
multivariable_models

['density_visceral_fat' 'density_bone' 'extrapulmonary_artery_volume'
 'extrapulmonary_vein_volume' 'a_diameter' 'heart_volume' 'airway_ratio'
 'ild_volume' 'age' 'gender_cl_Male']


,,nobs,shapiro_resid_pvalue,metric_train,metric_cv_mean,metric_cv_std,coef_a_diameter,coef_age,coef_airway_ratio,coef_const,coef_density_bone,...,signif_age,signif_airway_ratio,signif_const,signif_density_bone,signif_density_visceral_fat,signif_extrapulmonary_artery_volume,signif_extrapulmonary_vein_volume,signif_gender_cl_Male,signif_heart_volume,signif_ild_volume
y,model_dfn,,,,,,,,,,,,,,,,,,,,,
resolved_pe,"(const, density_visceral_fat, density_bone, extrapulmonary_artery_volume, extrapulmonary_vein_volume, a_diameter, heart_volume, airway_ratio, ild_volume, age, gender_cl_Male)",44,NaN,0.840909,0.791111,0.130791,0.632231,1.278631,-1.726273,-1.829879,-0.023133,...,,,**,,,,,,,


In [102]:
logit_sm = sm.Logit(y, X[remaining_features]).fit(cov_type='HC3', disp=False)
logit_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            resolved_pe   No. Observations:                   44
Model:                          Logit   Df Residuals:                       34
Method:                           MLE   Df Model:                            9
Date:                Mon, 03 Jul 2023   Pseudo R-squ.:                  0.3814
Time:                        13:54:40   Log-Likelihood:                -17.840
converged:                       True   LL-Null:                       -28.841
Covariance Type:                  HC3   LLR p-value:                  0.008871
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
density_visceral_fat            -0.1929      0.375     -0.514      0.607      -0.929       0.543
density_bone                    -0.1828      0.415     -0.440      0.660      -0.997       0.631
extrapulmonary_artery_volume     0.1776      0.489      0.364      0.716      -0.780       1.135
extrapulmonary_vein_volume      -1.5948      0.547     -2.915      0.004      -2.667      -0.523
a_diameter                       1.0503      1.017      1.032      0.302      -0.944       3.045
heart_volume                     1.8267      0.878      2.081      0.037       0.106       3.547
airway_ratio                    -0.6967      0.570     -1.222      0.222      -1.814       0.421
ild_volume                       0.5117      0.438      1.169      0.243      -0.346       1.370
age                              0.5203      0.618      0.842      0.400      -0.691       1.732
gender_cl_Male                  -0.0765      0.493     -0.155      0.877      -1.043       0.890
================================================================================================
"""

In [104]:
feat, prog = backward_stepwise_selection(X[remaining_features], y, 0.1)
logit_sm = sm.Logit(y, X[feat]).fit(cov_type='HC3', disp=False)
logit_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            resolved_pe   No. Observations:                   44
Model:                          Logit   Df Residuals:                       41
Method:                           MLE   Df Model:                            2
Date:                Mon, 03 Jul 2023   Pseudo R-squ.:                  0.2007
Time:                        13:55:01   Log-Likelihood:                -23.052
converged:                       True   LL-Null:                       -28.841
Covariance Type:                  HC3   LLR p-value:                  0.003059
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
heart_volume     1.0074      0.440      2.287      0.022       0.144       1.871
airway_ratio    -1.2232      0.440     -2.782      0.005      -2.085      -0.361
age              0.6331      0.393      1.612      0.107      -0.137       1.403
================================================================================
"""

In [105]:
X_temp = X[feat]
logit = LogisticRegressionCV(cv=10, solver='liblinear', max_iter=10_000, scoring='roc_auc')
logit.fit(X_temp, y)

auc = cross_val_score(
    logit, 
    X_temp,
    y,
    scoring='roc_auc',
    cv=10
)

print(f"ROC AUC: {np.mean(auc):.3f} ({np.std(auc):.3f})")
print(auc)

ROC AUC: 0.867 (0.180)
[1.         0.83333333 0.5        0.66666667 1.         1.
 1.         0.66666667 1.         1.        ]


### sklearn built in methods

# Combine univariable and multivariable results

In [85]:
ols_results = pd.concat([univariate_results, backward_stepwise_results], axis=0)

fname = 'logit_results'
if USE_INITIAL:
    fname += '_initial'
else: 
    fname += '_all'
if USE_CLUSTERED_SE:
    fname += '_clustered'
else: 
    fname += '_robust'
    
ols_results.to_csv(f'../output/regressions/{fname}.csv')

NameError: name 'backward_stepwise_results' is not defined

# Quick ensemble try

In [ ]:
# create model instance
bst = XGBClassifier(n_estimators=50, max_depth=3, learning_rate=0.01, objective='binary:logistic')
# fit model
bst.fit(X[multivariable_model_dfns['composite2']], y)
# make predictions
# preds = bst.predict(X[multivariable_model_dfns['composite2']])
cv_scores = cross_val_score(bst, X[multivariable_model_dfns['composite2']], y, scoring='roc_auc', cv=5)
np.mean(cv_scores)

# ROC Curves

In [ ]:
multivariable_model_dfns['composite2']

In [ ]:
keys = ['body', 'cardio', 'controls', 'composite2']
titles = ['Body Composition', 'Cardiopulmonary', 'Demographic Controls', 'Composite']

fig, axs = plt.subplots(2, 2, figsize=(10, 10), sharex=True, sharey=True)

for i, ax in enumerate(axs.flatten()):
    
    feat = multivariable_model_dfns[keys[i]]
    X_temp = sm.add_constant(X[feat]).reset_index(drop=True)
    y_temp = y.copy().reset_index(drop=True)
    
    cv = StratifiedKFold(n_splits=5)
    classifier = LogisticRegression(
            random_state=SEED,
            fit_intercept=False,
            max_iter=10_000, 
            tol=0.000001,
            penalty=None, 
            solver='newton-cg',
    )
    
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    
    for fold, (train, test) in enumerate(cv.split(X_temp, y_temp)):
        classifier.fit(X_temp.loc[train, :], y[train])
        viz = RocCurveDisplay.from_estimator(
            classifier,
            X_temp.loc[test, :],
            y_temp[test],
            name=f"ROC fold {fold}",
            alpha=0.3,
            lw=1,
            ax=ax,
        )
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)
    ax.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
    
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(
        mean_fpr,
        mean_tpr,
        color="b",
        label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )
    
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )
    
    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title=titles[i],
    )
    ax.axis("square")
    ax.legend(loc="lower right", fontsize=8)

plt.tight_layout()

plt.savefig('../figures/roc_curves.png')

In [ ]:
fname = 'roc_curve'
plot_title = 'ROC AUC'
if USE_INITIAL:
    fname += '_initial'
    plot_title += ' (Initial scans, '
else: 
    fname += '_all'
    plot_title += ' (All scans, '
if USE_CLUSTERED_SE:
    fname += '_clustered'
    plot_title += 'clustered SEs)'
else: 
    fname += '_robust'
    plot_title += 'robust SEs)'


feat = composite_model2

X_temp = sm.add_constant(X[feat]).reset_index(drop=True)
y_temp = y.copy().reset_index(drop=True)

cv = StratifiedKFold(n_splits=5)
classifier = LogisticRegression(
        random_state=SEED,
        fit_intercept=False,
        max_iter=10_000, 
        tol=0.000001,
        penalty=None, 
        solver='newton-cg',
)

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

fig, ax = plt.subplots(figsize=(6, 6))
for fold, (train, test) in enumerate(cv.split(X_temp, y_temp)):
    classifier.fit(X_temp.loc[train, :], y[train])
    viz = RocCurveDisplay.from_estimator(
        classifier,
        X_temp.loc[test, :],
        y_temp[test],
        name=f"ROC fold {fold}",
        alpha=0.3,
        lw=1,
        ax=ax,
    )
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
ax.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(
    mean_fpr,
    mean_tpr,
    color="b",
    label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(
    mean_fpr,
    tprs_lower,
    tprs_upper,
    color="grey",
    alpha=0.2,
    label=r"$\pm$ 1 std. dev.",
)

ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title=f"{plot_title}\n(Positive label=Resolved)",
)
ax.axis("square")
ax.legend(loc="lower right")

plt.savefig(f'../figures/{fname}.png')

plt.show()